# VacationPy
---

## Starter Code to Import Libraries and Load the Weather and Coordinates Data

In [36]:
# Dependencies and Setup
import hvplot.pandas
import pandas as pd
import requests
import pprint

# Import API key
from api_keys import geoapify_key

In [37]:
# Load the CSV file created in Part 1 into a Pandas DataFrame
city_data_df = pd.read_csv("output_data/cities.csv")

# Display sample data
city_data_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,rikitea,-23.1203,-134.9692,25.64,80,100,12.76,PF,1678606487
1,1,mar del plata,-38.0023,-57.5575,23.94,99,100,0.00,AR,1678606304
2,2,khatanga,71.9667,102.5000,-15.00,95,100,7.75,RU,1678606488
3,3,thompson,55.7435,-97.8558,-19.91,84,0,0.00,CA,1678606225
4,4,catuday,16.2923,119.8062,25.55,67,7,5.70,PH,1678606490


---
### Step 1: Create a map that displays a point for every city in the `city_data_df` DataFrame. The size of the point should be the humidity in each city.

In [38]:
%%capture --no-display

# Configure the map plot
map_plot_1=city_data_df.hvplot.points(
    "Lng", 
    "Lat",
    size="Humidity",
    color="City",
    geo=True,
    tiles="OSM",
    )

# Display the map
map_plot_1

:Overlay
   .Tiles.I  :Tiles   [x,y]
   .Points.I :Points   [Lng,Lat]   (City,Humidity)

### Step 2: Narrow down the `city_data_df` DataFrame to find your ideal weather condition

In [39]:
# Narrow down cities that fit criteria and drop any results with null values
citydata=city_data_df.loc[(city_data_df["Max Temp"]<=27)&
                               (city_data_df["Wind Speed"]<4.5)&
                               (city_data_df["Cloudiness"]==0)]

# Drop any rows with null values
citydataclean=citydata.dropna(how='any')

# Display sample data
citydataclean.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
3,3,thompson,55.7435,-97.8558,-19.91,84,0,0.00,CA,1678606225
11,11,yellowknife,62.4560,-114.3525,-28.71,76,0,2.06,CA,1678606217
15,15,comodoro rivadavia,-45.8667,-67.5000,19.92,56,0,1.03,AR,1678606203
22,22,coquimbo,-29.9533,-71.3436,16.85,100,0,1.54,CL,1678606390
25,25,tasiilaq,65.6145,-37.6368,-12.91,59,0,1.92,GL,1678606503


### Step 3: Create a new DataFrame called `hotel_df`.

In [40]:
# Use the Pandas copy function to create DataFrame called hotel_df to store the city, country, coordinates, and humidity
hotel_df= citydataclean[['City','Country','Lat','Lng','Humidity']].copy()

# Add an empty column, "Hotel Name," to the DataFrame so you can store the hotel found using the Geoapify API
hotel_df["Hotel Name"] = ""

# Display sample data
hotel_df.head()

,City,Country,Lat,Lng,Humidity,Hotel Name
3,thompson,CA,55.7435,-97.8558,84,
11,yellowknife,CA,62.4560,-114.3525,76,
15,comodoro rivadavia,AR,-45.8667,-67.5000,56,
22,coquimbo,CL,-29.9533,-71.3436,100,
25,tasiilaq,GL,65.6145,-37.6368,59,


### Step 4: For each city, use the Geoapify API to find the first hotel located within 10,000 metres of your coordinates.

In [41]:
# Set parameters to search for a hotel
radius = 10000
params = {
    "categories":"accommodation.hotel",
    "apiKey":geoapify_key,
    "limit":10
}
# Print a message to follow up the hotel search
print("Starting hotel search")

# Iterate through the hotel_df DataFrame
for index, row in hotel_df.iterrows():
    # get latitude, longitude from the DataFrame
    latitude=row["Lat"]
    longitude=row["Lng"]
    
    # Add filter and bias parameters with the current city's latitude and longitude to the params dictionary
    params["filter"] = f"circle:{longitude},{latitude},{radius}"
    params["bias"] = f"proximity:{longitude},{latitude}"
    
    # Set base URL
    base_url = "https://api.geoapify.com/v2/places"


    # Make and API request using the params dictionaty
    name_address = requests.get(base_url, params=params)
    
    # Convert the API response to JSON format
    name_address = name_address.json()
    
    # Grab the first hotel from the results and store the name in the hotel_df DataFrame
    try:
        hotel_df.loc[index, "Hotel Name"] = name_address["features"][0]["properties"]["name"]
    except (KeyError, IndexError):
        # If no hotel is found, set the hotel name as "No hotel found".
        hotel_df.loc[index, "Hotel Name"] = "No hotel found"
        
    # Log the search results
    print(f"{hotel_df.loc[index, 'City']} - nearest hotel: {hotel_df.loc[index, 'Hotel Name']}")

# Display sample data
hotel_df

Starting hotel search
thompson - nearest hotel: Thompson Inn
yellowknife - nearest hotel: No hotel found
comodoro rivadavia - nearest hotel: No hotel found
coquimbo - nearest hotel: Hotel Iberia
tasiilaq - nearest hotel: Angmagssalik
bethel - nearest hotel: Hampton Inn Danbury
san antonio - nearest hotel: Riverwalk Plaza Hotel and Suites
saldanha - nearest hotel: Hoedjesbaai
cape town - nearest hotel: Townhouse Hotel
acapulco - nearest hotel: Hotel del Valle
lowell - nearest hotel: UMass Lowell Inn and Conference Center
upington - nearest hotel: Die Eiland Holiday Ressort
ypsonas - nearest hotel: Club Aphrodite
westport - nearest hotel: Norwalk Inn and Conference Center
puerto escondido - nearest hotel: Hotel Barlovento
cabo san lucas - nearest hotel: Comfort Rooms
kihei - nearest hotel: Maui Beach Resort
mount gambier - nearest hotel: The Old Mount Gambier Gaol
turkmenabat - nearest hotel: Yupek Yoly
puerto padre - nearest hotel: Hostal El Paseo
homer - nearest hotel: Driftwood Inn & 

,City,Country,Lat,Lng,Humidity,Hotel Name
3,thompson,CA,55.7435,-97.8558,84,Thompson Inn
11,yellowknife,CA,62.4560,-114.3525,76,No hotel found
15,comodoro rivadavia,AR,-45.8667,-67.5000,56,No hotel found
22,coquimbo,CL,-29.9533,-71.3436,100,Hotel Iberia
25,tasiilaq,GL,65.6145,-37.6368,59,Angmagssalik
52,bethel,US,41.3712,-73.4140,72,Hampton Inn Danbury
62,san antonio,US,29.4241,-98.4936,84,Riverwalk Plaza Hotel and Suites
82,saldanha,ZA,-33.0117,17.9442,59,Hoedjesbaai
86,cape town,ZA,-33.9258,18.4232,59,Townhouse Hotel
97,acapulco,MX,16.8634,-99.8901,83,Hotel del Valle


### Step 5: Add the hotel name and the country as additional information in the hover message for each city in the map.

In [42]:
%%capture --no-display

# Configure the map plot
map_plot_2=hotel_df.hvplot.points(
    "Lng", 
    "Lat",
    size="Humidity",
    color="City",
    geo=True,
    tiles="OSM",
    hover_cols=["City","Country"]
    )

# Display the map
map_plot_2

:Overlay
   .Tiles.I  :Tiles   [x,y]
   .Points.I :Points   [Lng,Lat]   (City,Humidity,Country)